In [1]:
#from my_snn.rsnn import RSNN, RSNN_2l, RSNN_2l_ALIF
from my_snn.rsnn_delays import RSNN_d_d, RSNN_d_i, RSNN_d_all
from my_snn.tonic_dataloader import DatasetLoader
from my_snn.utils import train, training_plots, ModelLoader
import torch
device = "cuda:0"

import matplotlib.pyplot as  plt

time_window = 250
batch_size = 256
dataset = 'shd'

DL = DatasetLoader(dataset=dataset, caching='disk', num_workers=0, batch_size=batch_size, time_window=time_window)
test_loader, train_loader = DL.get_dataloaders()

ckpt_dir = 'shd_soa_tests'

data = test_loader, train_loader

dropout = 0.2

import numpy as np
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        snn.test(test_loader, dropout=dropout)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)

D:\RESEARCHERS\Alberto\Anaconda3\envs\deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hidden = (32, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=32, bias=False)
  (f1_f2): Linear(in_features=1600, out_features=32, bias=False)
  (f2_o): Linear(in_features=1600, out_features=20, bias=False)
)
Total params: 105600
Total mult-adds (M): 26.4
training shd250_32x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 27.81616
Step [20/31], Loss: 21.00935
Step [30/31], Loss: 13.52243
Time elasped: 120.80301260948181
Epoch [2/50]
Step [10/31], Loss: 8.70263
Step [20/31], Loss: 7.13514
Step [30/31], Loss: 5.55678
Time elasped: 52.84698724746704
Epoch [3/50]
Step [10/31], Loss: 5.01566
Step [20/31], Loss: 3.65757
Step [30/31], Loss: 4.04503
Time elasped: 50.9273898601532
Epoch [4/50]
Step [10/31], Loss: 3.45301
Step [20/31], 

Step [30/31], Loss: 0.33015
Time elasped: 50.57301640510559
Epoch [49/50]
Step [10/31], Loss: 0.12665
Step [20/31], Loss: 0.05901
Step [30/31], Loss: 0.08977
Time elasped: 50.527451276779175
Epoch [50/50]
Step [10/31], Loss: 0.06985
Step [20/31], Loss: 0.05915
Step [30/31], Loss: 0.08015
Time elasped: 50.218403577804565
Test Loss: 1.0251699941498893
Avg spk_count per neuron for all 250 timesteps 23.13970947265625
Test Accuracy of the model on the test samples: 84.229


In [3]:
snn = ModelLoader('shd250_32x2_d_150_3_fs_88330078125', ckpt_dir, batch_size, device)
snn.test(test_loader, dropout = dropout)

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 0.7476942411490849
Avg spk_count per neuron for all 250 timesteps 22.984664916992188
Test Accuracy of the model on the test samples: 88.135


In [4]:
k = 1
snn_pooled = ModelLoader('shd250_32x2_d_150_3_fs_88330078125', ckpt_dir, batch_size, 'cpu')
snn_pooled.pool_delays('ho', k=k)
snn_pooled.test(test_loader, dropout = dropout)

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 3.337204558508737
Avg spk_count per neuron for all 250 timesteps 11.622993469238281
Test Accuracy of the model on the test samples: 15.430


In [5]:
k = 5
snn_pooled = ModelLoader('shd250_32x2_d_150_3_fs_88330078125', ckpt_dir, batch_size, 'cpu')
snn_pooled.pool_delays('ho', k=k)
snn_pooled.test(test_loader, dropout = dropout)

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 2.0916478633880615
Avg spk_count per neuron for all 250 timesteps 16.021957397460938
Test Accuracy of the model on the test samples: 43.701


In [6]:
k = 10
snn_pooled = ModelLoader('shd250_32x2_d_150_3_fs_88330078125', ckpt_dir, batch_size, 'cpu')
snn_pooled.pool_delays('ho', k=k)
snn_pooled.test(test_loader, dropout = dropout)

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 1.2018194879804338
Avg spk_count per neuron for all 250 timesteps 18.935958862304688
Test Accuracy of the model on the test samples: 64.941


In [7]:
k = 15
snn_pooled = ModelLoader('shd250_32x2_d_150_3_fs_88330078125', ckpt_dir, batch_size, 'cpu')
snn_pooled.pool_delays('ho', k=k)
snn_pooled.test(test_loader, dropout = dropout)

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 1.1170856526919775
Avg spk_count per neuron for all 250 timesteps 20.339523315429688
Test Accuracy of the model on the test samples: 71.729


In [11]:
dropout = 0.2

import numpy as np
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        #k = 50-epoch
        #print(f'pruninng to {k} delays')
        snn.pool_delays('ho', k=k)
        snn.test(test_loader, dropout = dropout)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)

In [15]:
k = 15
snn_pooled = ModelLoader('shd250_32x2_d_150_3_fs_88330078125', ckpt_dir, batch_size, device)
snn_pooled.pool_delays('ho', k=k, freeze=False)
snn_pooled.test(test_loader, dropout = dropout)

snn_pooled.train_loss = []
snn_pooled.test_loss = []

snn_pooled.modelname = f'shd250_64x2_d_150_3_fs_pooled_{k}'
train(snn_pooled, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
torch.cuda.empty_cache() 

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 1.1005526525633675
Avg spk_count per neuron for all 250 timesteps 20.295928955078125
Test Accuracy of the model on the test samples: 71.826
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=32, bias=False)
  (f1_f2): Linear(in_features=1600, out_features=32, bias=False)
  (f2_o): Linear(in_features=1600, out_features=20, bias=False)
)
Total params: 105600
Total mult-adds (M): 26.4
training shd250_64x2_d_150_3_fs_pooled_15 for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 2.71374
Step [20/31], Loss: 1.98415
Step [30/31], Loss: 1.65509
Time elasped: 50.311426639556885
Epoch [2/50]
Step [10/31], Loss: 1.52445
Step [20/31], Loss: 1.29562
Step [30/31], Lo

Step [10/31], Loss: 0.06875
Step [20/31], Loss: 0.06266
Step [30/31], Loss: 0.08056
Time elasped: 49.69226884841919
Epoch [47/50]
Step [10/31], Loss: 0.05412
Step [20/31], Loss: 0.09932
Step [30/31], Loss: 0.08156
Time elasped: 49.78353548049927
Epoch [48/50]
Step [10/31], Loss: 0.12476
Step [20/31], Loss: 0.12656
Step [30/31], Loss: 0.08789
Time elasped: 49.75903797149658
Epoch [49/50]
Step [10/31], Loss: 0.07375
Step [20/31], Loss: 0.06792
Step [30/31], Loss: 0.06378
Time elasped: 50.29643940925598
Epoch [50/50]
Step [10/31], Loss: 0.09644
Step [20/31], Loss: 0.10046
Step [30/31], Loss: 0.09119
Time elasped: 50.785301208496094
Test Loss: 1.0371268221310206
Avg spk_count per neuron for all 250 timesteps 20.283554077148438
Test Accuracy of the model on the test samples: 84.180


In [16]:
k = 20
snn_pooled = ModelLoader('shd250_32x2_d_150_3_fs_88330078125', ckpt_dir, batch_size, device)
snn_pooled.pool_delays('ho', k=k, freeze=False)
snn_pooled.test(test_loader, dropout = dropout)

snn_pooled.train_loss = []
snn_pooled.test_loss = []

snn_pooled.modelname = f'shd250_64x2_d_150_3_fs_pooled_{k}'
train(snn_pooled, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
torch.cuda.empty_cache() 

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 1.0376147202083044
Avg spk_count per neuron for all 250 timesteps 21.060394287109375
Test Accuracy of the model on the test samples: 78.076
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=32, bias=False)
  (f1_f2): Linear(in_features=1600, out_features=32, bias=False)
  (f2_o): Linear(in_features=1600, out_features=20, bias=False)
)
Total params: 105600
Total mult-adds (M): 26.4
training shd250_64x2_d_150_3_fs_pooled_20 for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 1.20988
Step [20/31], Loss: 1.15393
Step [30/31], Loss: 0.69571
Time elasped: 50.47208523750305
Epoch [2/50]
Step [10/31], Loss: 0.60727
Step [20/31], Loss: 0.64239
Step [30/31], Los

Step [10/31], Loss: 0.05115
Step [20/31], Loss: 0.04100
Step [30/31], Loss: 0.04196
Time elasped: 49.65607500076294
Epoch [47/50]
Step [10/31], Loss: 0.03568
Step [20/31], Loss: 0.04125
Step [30/31], Loss: 0.03388
Time elasped: 50.03991627693176
Epoch [48/50]
Step [10/31], Loss: 0.03777
Step [20/31], Loss: 0.04999
Step [30/31], Loss: 0.06518
Time elasped: 49.99459385871887
Epoch [49/50]
Step [10/31], Loss: 0.06869
Step [20/31], Loss: 0.05710
Step [30/31], Loss: 0.09606
Time elasped: 49.75317978858948
Epoch [50/50]
Step [10/31], Loss: 0.07198
Step [20/31], Loss: 0.05387
Step [30/31], Loss: 0.08296
Time elasped: 49.60433006286621
Test Loss: 1.1723269649914332
Avg spk_count per neuron for all 250 timesteps 22.472816467285156
Test Accuracy of the model on the test samples: 83.594


In [17]:
k = 25
snn_pooled = ModelLoader('shd250_32x2_d_150_3_fs_88330078125', ckpt_dir, batch_size, device)
snn_pooled.pool_delays('ho', k=k, freeze=False)
snn_pooled.test(test_loader, dropout = dropout)

snn_pooled.train_loss = []
snn_pooled.test_loss = []

snn_pooled.modelname = f'shd250_64x2_d_150_3_fs_pooled_{k}'
train(snn_pooled, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
torch.cuda.empty_cache() 

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 1.087275207042694
Avg spk_count per neuron for all 250 timesteps 21.723007202148438
Test Accuracy of the model on the test samples: 80.029
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=32, bias=False)
  (f1_f2): Linear(in_features=1600, out_features=32, bias=False)
  (f2_o): Linear(in_features=1600, out_features=20, bias=False)
)
Total params: 105600
Total mult-adds (M): 26.4
training shd250_64x2_d_150_3_fs_pooled_25 for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.81072
Step [20/31], Loss: 0.51156
Step [30/31], Loss: 0.47925
Time elasped: 50.527010440826416
Epoch [2/50]
Step [10/31], Loss: 0.27241
Step [20/31], Loss: 0.26811
Step [30/31], Los

Step [10/31], Loss: 0.02280
Step [20/31], Loss: 0.02320
Step [30/31], Loss: 0.01483
Time elasped: 49.824939012527466
Epoch [47/50]
Step [10/31], Loss: 0.03652
Step [20/31], Loss: 0.03923
Step [30/31], Loss: 0.02354
Time elasped: 49.735639333724976
Epoch [48/50]
Step [10/31], Loss: 0.03780
Step [20/31], Loss: 0.03084
Step [30/31], Loss: 0.06137
Time elasped: 49.76849675178528
Epoch [49/50]
Step [10/31], Loss: 0.04053
Step [20/31], Loss: 0.03933
Step [30/31], Loss: 0.05000
Time elasped: 50.01298213005066
Epoch [50/50]
Step [10/31], Loss: 0.03247
Step [20/31], Loss: 0.02750
Step [30/31], Loss: 0.02155
Time elasped: 49.574827432632446
Test Loss: 1.307479407106127
Avg spk_count per neuron for all 250 timesteps 23.20374298095703
Test Accuracy of the model on the test samples: 82.080


In [18]:
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        snn.test(test_loader, dropout=dropout)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)
            
hidden = (32, 2, 'f')
depth, stride = 150, 1
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
torch.cuda.empty_cache() 

delays: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=32, bias=False)
  (f1_f2): Linear(in_features=4800, out_features=32, bias=False)
  (f2_o): Linear(in_features=4800, out_features=20, bias=False)
)
Total params: 272000
Total mult-adds (M): 68.0
training shd250_32x2_d_150_1_fs for 50 epochs...
Epoch [1/50]
Step

Step [10/31], Loss: 0.35216
Step [20/31], Loss: 0.29970
Step [30/31], Loss: 0.24087
Time elasped: 52.812596559524536
Epoch [47/50]
Step [10/31], Loss: 0.24894
Step [20/31], Loss: 0.23552
Step [30/31], Loss: 0.14829
Time elasped: 53.087034463882446
Epoch [48/50]
Step [10/31], Loss: 0.20508
Step [20/31], Loss: 0.22659
Step [30/31], Loss: 0.23649
Time elasped: 52.727020263671875
Epoch [49/50]
Step [10/31], Loss: 0.16875
Step [20/31], Loss: 0.10526
Step [30/31], Loss: 0.10080
Time elasped: 52.72310972213745
Epoch [50/50]
Step [10/31], Loss: 0.25720
Step [20/31], Loss: 0.27514
Step [30/31], Loss: 0.35196
Time elasped: 52.70770883560181
Test Loss: 0.9776796783719744
Avg spk_count per neuron for all 250 timesteps 14.183151245117188
Test Accuracy of the model on the test samples: 84.277
saving max acc: 84.27734375


### 86% (150,6)

In [19]:
hidden = (32, 2, 'f')
depth, stride = 150, 6
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
torch.cuda.empty_cache() 

delays: [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=32, bias=False)
  (f1_f2): Linear(in_features=800, out_features=32, bias=False)
  (f2_o): Linear(in_features=800, out_features=20, bias=False)
)
Total params: 64000
Total mult-adds (M): 16.0
training shd250_32x2_d_150_6_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 28.72695
Step [20/31], Loss: 21.85493
Step [30/31], Loss: 14.47940
Time elasped: 50.37055492401123
Epoch [2/50]
Step [10/31], Loss: 10.09730
Step [20/31], Loss: 7.51193
Step [30/31], Loss: 6.28766
Time elasped: 49.91483926773071
Epoch [3/50]
Step [10/31], Loss: 5.19257
Step [20/31], Loss: 4.06005
Step [30/31], Loss: 3.72725
Time elasped: 49.62849473953247
Epoch [4/50]
Step [10/31], Loss: 3.05449
Step [20/31], Loss: 2.92667
Step [30/31], Loss: 2.34683
Time elasped: 49.95435547828674
Epoch [5/50]
Step [10/31], Loss: 2

Step [30/31], Loss: 0.03196
Time elasped: 49.995790004730225
Epoch [50/50]
Step [10/31], Loss: 0.08095
Step [20/31], Loss: 0.09304
Step [30/31], Loss: 0.05832
Time elasped: 49.88288140296936
Test Loss: 0.7189846038818359
Avg spk_count per neuron for all 250 timesteps 30.32708740234375
Test Accuracy of the model on the test samples: 86.621


In [ ]:
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        snn.test(test_loader, dropout=dropout)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)

k = 10
snn_pooled = ModelLoader('shd250_32x2_d_150_6_fs_8681640625', ckpt_dir, batch_size, 'cuda:0')
snn_pooled.pool_delays('ho', k=k)
snn_pooled.test(test_loader, dropout = dropout)

snn_pooled.train_loss = []
snn_pooled.test_loss = []

snn_pooled.modelname = f'shd250_32x2_d_150_6_fs_pooled_{k}'
train(snn_pooled, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
torch.cuda.empty_cache() 

delays: [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 0.9310136267117092
Avg spk_count per neuron for all 250 timesteps 27.26372528076172
Test Accuracy of the model on the test samples: 77.295
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=32, bias=False)
  (f1_f2): Linear(in_features=800, out_features=32, bias=False)
  (f2_o): Linear(in_features=800, out_features=20, bias=False)
)
Total params: 64000
Total mult-adds (M): 16.0
training shd250_32x2_d_150_6_fs_pooled_10 for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 1.33637
Step [20/31], Loss: 0.90231
Step [30/31], Loss: 0.81340
Time elasped: 62.18956542015076
Epoch [2/50]
Step [10/31], Loss: 0.68072
Step [20/31], Loss: 0.71645
Step [30/31], Loss: 0.72033
Time elasped: 49.533687353134155
Epoch [3/50]
Step [10/31], Loss: 0.60363
Step [20/31], Loss: 0.658